In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from networks import NoKafnet, Kafnet
import utils.datasetsUtils.CIFAR as CIFAR
from utils.datasetsUtils.taskManager import SingleTargetClassificationTask, NoTask
from configs.configs import DefaultConfig, CONFIG_CIFAR10
from torchvision.transforms import transforms
import torch
from Trainer import Trainer
import matplotlib.pyplot as plt
from collections import defaultdict
import copy
import numpy as np

In [16]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

True
1
GeForce GTX 1050


In [4]:
config = CONFIG_CIFAR10()
print(config)

confing_no_ewt = copy.copy(config)
confing_no_ewt.USE_EWC = False
print(confing_no_ewt)

CONFIG PARAMETERS
BATCH_SIZE: 64
DEVICE: cuda
EPOCHS: 15
EWC_IMPORTANCE: 1000
EWC_SAMPLE_SIZE: 250
IS_CONVOLUTIONAL: True
ITERS: 1
L1_REG: 0
LOSS: cross_entropy
LR: 0.001
MODEL_NAME: 
OPTIMIZER: SGD
RUN_NAME: default
USE_EWC: True
USE_TENSORBOARD: True

CONFIG PARAMETERS
BATCH_SIZE: 64
DEVICE: cuda
EPOCHS: 15
EWC_IMPORTANCE: 1000
EWC_SAMPLE_SIZE: 250
IS_CONVOLUTIONAL: True
ITERS: 1
L1_REG: 0
LOSS: cross_entropy
LR: 0.001
MODEL_NAME: 
OPTIMIZER: SGD
RUN_NAME: default
USE_EWC: False
USE_TENSORBOARD: True



In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
)

dataset = CIFAR.Cifar10('../data/cifar10', SingleTargetClassificationTask(), download=True,
                        force_download=False, train_split=0.8, transform=transform, target_transform=None)
dataset.load_dataset()

dataset_no_ewt = copy.deepcopy(dataset)
dataset_kaf = copy.deepcopy(dataset)
dataset_kaf_no_ewt = copy.deepcopy(dataset)

../data/cifar10/download
task #0 with train 48000 and test 12000 images (label: airplane)
task #1 with train 48000 and test 12000 images (label: automobile)
task #2 with train 48000 and test 12000 images (label: bird)
task #3 with train 48000 and test 12000 images (label: cat)
task #4 with train 48000 and test 12000 images (label: deer)
task #5 with train 48000 and test 12000 images (label: dog)
task #6 with train 48000 and test 12000 images (label: frog)
task #7 with train 48000 and test 12000 images (label: horse)
task #8 with train 48000 and test 12000 images (label: ship)
task #9 with train 48000 and test 12000 images (label: truck)


In [6]:
net_ewt = NoKafnet.CNN(dataset.tasks_number)
net_no_ewt = NoKafnet.CNN(dataset.tasks_number)

net_kaf_ewt = Kafnet.KAFCNN(dataset.tasks_number)
net_kaf_no_ewt = Kafnet.KAFCNN(dataset.tasks_number)

print('Numero di parametri rete classica: ', sum([torch.numel(p) for p in net_ewt.parameters()]))
print('Numero di parametri KAFNET: ', sum([torch.numel(p) for p in net_kaf_ewt.parameters()]))

Numero di parametri rete classica:  171732
Numero di parametri KAFNET:  120020


In [7]:
trainer_ewt = Trainer(net_ewt, dataset, config)
trainer_no_ewt = Trainer(net_no_ewt, dataset_no_ewt, confing_no_ewt)

trainer_kaf_ewt = Trainer(net_kaf_ewt, dataset_kaf, config)
trainer_kaf_no_ewt = Trainer(net_kaf_no_ewt, dataset_kaf_no_ewt, confing_no_ewt)

In [8]:
results_ewt = trainer_ewt.all_tasks()

Training task 0, epoch 1: 100%|██████████| 750/750 [00:22<00:00, 33.94it/s, loss=0.276, batch#=750]
Testing task 0: 188it [00:03, 53.62it/s, batch#=188]                         
Training task 0, epoch 2: 100%|██████████| 750/750 [00:21<00:00, 38.97it/s, loss=0.235, batch#=750]
Testing task 0: 188it [00:02, 65.67it/s, batch#=188]                         
Training task 0, epoch 3: 100%|██████████| 750/750 [00:18<00:00, 41.68it/s, loss=0.223, batch#=750]
Testing task 0: 188it [00:02, 63.79it/s, batch#=188]                         
Training task 0, epoch 4: 100%|██████████| 750/750 [00:20<00:00, 37.41it/s, loss=0.214, batch#=750]
Testing task 0: 188it [00:03, 59.82it/s, batch#=188]                         
Training task 0, epoch 5: 100%|██████████| 750/750 [00:20<00:00, 36.06it/s, loss=0.208, batch#=750]
Testing task 0: 188it [00:03, 57.87it/s, batch#=188]                         
Training task 0, epoch 6: 100%|██████████| 750/750 [00:20<00:00, 36.63it/s, loss=0.203, batch#=750]
Testing ta

Training task 2, epoch 5: 100%|██████████| 750/750 [00:23<00:00, 31.33it/s, loss=0.335, batch#=750]
Testing task 2: 188it [00:03, 59.36it/s, batch#=188]                         
Testing task 0: 188it [00:03, 59.32it/s, batch#=188]                         
Testing task 1: 188it [00:03, 60.48it/s, batch#=188]                         
Training task 2, epoch 6: 100%|██████████| 750/750 [00:23<00:00, 32.36it/s, loss=0.334, batch#=750]
Testing task 2: 188it [00:03, 54.13it/s, batch#=188]                         
Testing task 0: 188it [00:03, 55.32it/s, batch#=188]                         
Testing task 1: 188it [00:03, 53.45it/s, batch#=188]                         
Training task 2, epoch 7: 100%|██████████| 750/750 [00:24<00:00, 31.13it/s, loss=0.333, batch#=750]
Testing task 2: 188it [00:03, 63.94it/s, batch#=188]                         
Testing task 0: 188it [00:03, 61.20it/s, batch#=188]                         
Testing task 1: 188it [00:03, 55.80it/s, batch#=188]                        

Testing task 2: 188it [00:02, 70.37it/s, batch#=188]                         
Training task 3, epoch 12: 100%|██████████| 750/750 [00:18<00:00, 42.13it/s, loss=0.32, batch#=750] 
Testing task 3: 188it [00:02, 73.99it/s, batch#=188]                         
Testing task 0: 188it [00:02, 72.51it/s, batch#=188]                         
Testing task 1: 188it [00:02, 72.09it/s, batch#=188]                         
Testing task 2: 188it [00:02, 72.64it/s, batch#=188]                         
Training task 3, epoch 13: 100%|██████████| 750/750 [00:17<00:00, 42.05it/s, loss=0.32, batch#=750] 
Testing task 3: 188it [00:02, 72.06it/s, batch#=188]                         
Testing task 0: 188it [00:02, 70.16it/s, batch#=188]                         
Testing task 1: 188it [00:02, 70.33it/s, batch#=188]                         
Testing task 2: 188it [00:02, 71.24it/s, batch#=188]                         
Training task 3, epoch 14: 100%|██████████| 750/750 [00:18<00:00, 41.58it/s, loss=0.32, batch#=7

Training task 4, epoch 14: 100%|██████████| 750/750 [00:20<00:00, 36.62it/s, loss=0.373, batch#=750]
Testing task 4: 188it [00:03, 60.24it/s, batch#=188]                         
Testing task 0: 188it [00:03, 54.21it/s, batch#=188]                         
Testing task 1: 188it [00:03, 55.71it/s, batch#=188]                         
Testing task 2: 188it [00:03, 54.95it/s, batch#=188]                         
Testing task 3: 188it [00:03, 58.96it/s, batch#=188]                         
Training task 4, epoch 15: 100%|██████████| 750/750 [00:20<00:00, 35.92it/s, loss=0.372, batch#=750]
Testing task 4: 188it [00:03, 60.33it/s, batch#=188]                         
Testing task 0: 188it [00:03, 59.82it/s, batch#=188]                         
Testing task 1: 188it [00:03, 59.88it/s, batch#=188]                         
Testing task 2: 188it [00:03, 58.85it/s, batch#=188]                         
Testing task 3: 188it [00:03, 58.19it/s, batch#=188]                         
Training task 5, e

Testing task 2: 188it [00:03, 61.10it/s, batch#=188]                         
Testing task 3: 188it [00:03, 61.04it/s, batch#=188]                         
Testing task 4: 188it [00:03, 61.27it/s, batch#=188]                         
Training task 5, epoch 14: 100%|██████████| 750/750 [00:20<00:00, 36.60it/s, loss=0.352, batch#=750]
Testing task 5: 188it [00:03, 60.91it/s, batch#=188]                         
Testing task 0: 188it [00:03, 59.31it/s, batch#=188]                         
Testing task 1: 188it [00:03, 57.86it/s, batch#=188]                         
Testing task 2: 188it [00:03, 58.64it/s, batch#=188]                         
Testing task 3: 188it [00:03, 58.09it/s, batch#=188]                         
Testing task 4: 188it [00:03, 57.81it/s, batch#=188]                         
Training task 5, epoch 15: 100%|██████████| 750/750 [00:20<00:00, 35.96it/s, loss=0.35, batch#=750] 
Testing task 5: 188it [00:03, 60.52it/s, batch#=188]                         
Testing task 0: 18

Testing task 2: 188it [00:03, 58.25it/s, batch#=188]                         
Testing task 3: 188it [00:03, 57.98it/s, batch#=188]                         
Testing task 4: 188it [00:03, 58.97it/s, batch#=188]                         
Testing task 5: 188it [00:03, 58.85it/s, batch#=188]                         
Training task 6, epoch 12: 100%|██████████| 750/750 [00:21<00:00, 35.13it/s, loss=0.294, batch#=750]
Testing task 6: 188it [00:03, 60.16it/s, batch#=188]                         
Testing task 0: 188it [00:03, 59.15it/s, batch#=188]                         
Testing task 1: 188it [00:03, 58.90it/s, batch#=188]                         
Testing task 2: 188it [00:03, 57.30it/s, batch#=188]                         
Testing task 3: 188it [00:03, 57.48it/s, batch#=188]                         
Testing task 4: 188it [00:03, 57.05it/s, batch#=188]                         
Testing task 5: 188it [00:03, 53.86it/s, batch#=188]                         
Training task 6, epoch 13: 100%|█████████

Testing task 0: 188it [00:02, 68.66it/s, batch#=188]                         
Testing task 1: 188it [00:02, 72.66it/s, batch#=188]                         
Testing task 2: 188it [00:02, 72.15it/s, batch#=188]                         
Testing task 3: 188it [00:02, 67.85it/s, batch#=188]                         
Testing task 4: 188it [00:02, 65.89it/s, batch#=188]                         
Testing task 5: 188it [00:02, 70.47it/s, batch#=188]                         
Testing task 6: 188it [00:02, 72.35it/s, batch#=188]                         
Training task 7, epoch 9: 100%|██████████| 750/750 [00:17<00:00, 41.70it/s, loss=0.371, batch#=750]
Testing task 7: 188it [00:02, 73.32it/s, batch#=188]                         
Testing task 0: 188it [00:02, 74.30it/s, batch#=188]                         
Testing task 1: 188it [00:02, 72.56it/s, batch#=188]                         
Testing task 2: 188it [00:02, 73.40it/s, batch#=188]                         
Testing task 3: 188it [00:02, 72.30it/s, b

Testing task 8: 188it [00:02, 73.90it/s, batch#=188]                         
Testing task 0: 188it [00:02, 71.00it/s, batch#=188]                         
Testing task 1: 188it [00:02, 72.23it/s, batch#=188]                         
Testing task 2: 188it [00:03, 62.30it/s, batch#=188]                         
Testing task 3: 188it [00:03, 58.58it/s, batch#=188]                         
Testing task 4: 188it [00:03, 59.08it/s, batch#=188]                         
Testing task 5: 188it [00:03, 60.23it/s, batch#=188]                         
Testing task 6: 188it [00:03, 61.06it/s, batch#=188]                         
Testing task 7: 188it [00:02, 66.00it/s, batch#=188]                         
Training task 8, epoch 5: 100%|██████████| 750/750 [00:17<00:00, 41.83it/s, loss=0.281, batch#=750]
Testing task 8: 188it [00:02, 65.87it/s, batch#=188]                         
Testing task 0: 188it [00:02, 70.90it/s, batch#=188]                         
Testing task 1: 188it [00:02, 72.87it/s, b

Testing task 1: 188it [00:02, 74.80it/s, batch#=188]                         
Testing task 2: 188it [00:02, 72.87it/s, batch#=188]                         
Testing task 3: 188it [00:02, 72.33it/s, batch#=188]                         
Testing task 4: 188it [00:02, 72.20it/s, batch#=188]                         
Testing task 5: 188it [00:02, 72.74it/s, batch#=188]                         
Testing task 6: 188it [00:02, 71.68it/s, batch#=188]                         
Testing task 7: 188it [00:02, 72.36it/s, batch#=188]                         
Training task 8, epoch 15: 100%|██████████| 750/750 [00:17<00:00, 42.25it/s, loss=0.271, batch#=750]
Testing task 8: 188it [00:02, 74.86it/s, batch#=188]                         
Testing task 0: 188it [00:02, 75.03it/s, batch#=188]                         
Testing task 1: 188it [00:02, 74.64it/s, batch#=188]                         
Testing task 2: 188it [00:02, 74.63it/s, batch#=188]                         
Testing task 3: 188it [00:02, 75.03it/s, 

Testing task 6: 188it [00:02, 71.72it/s, batch#=188]                         
Testing task 7: 188it [00:02, 72.07it/s, batch#=188]                         
Testing task 8: 188it [00:02, 71.91it/s, batch#=188]                         
Training task 9, epoch 9: 100%|██████████| 750/750 [00:17<00:00, 42.07it/s, loss=0.345, batch#=750]
Testing task 9: 188it [00:02, 73.04it/s, batch#=188]                         
Testing task 0: 188it [00:02, 74.00it/s, batch#=188]                         
Testing task 1: 188it [00:06, 28.93it/s, batch#=188]                         
Testing task 2: 188it [00:03, 61.37it/s, batch#=188]                         
Testing task 3: 188it [00:03, 61.79it/s, batch#=188]                         
Testing task 4: 188it [00:02, 62.77it/s, batch#=188]                         
Testing task 5: 188it [00:03, 62.89it/s, batch#=188]                         
Testing task 6: 188it [00:03, 62.65it/s, batch#=188]                         
Testing task 7: 188it [00:02, 64.03it/s, b

In [9]:
results_no_ewt = trainer_no_ewt.all_tasks()

Training task 0, epoch 1: 100%|██████████| 750/750 [00:19<00:00, 39.26it/s, loss=0.275, batch#=750]
Testing task 0: 188it [00:03, 58.64it/s, batch#=188]                         
Training task 0, epoch 2: 100%|██████████| 750/750 [00:19<00:00, 38.87it/s, loss=0.238, batch#=750]
Testing task 0: 188it [00:03, 61.84it/s, batch#=188]                         
Training task 0, epoch 3: 100%|██████████| 750/750 [00:18<00:00, 40.41it/s, loss=0.225, batch#=750]
Testing task 0: 188it [00:02, 65.06it/s, batch#=188]                         
Training task 0, epoch 4: 100%|██████████| 750/750 [00:18<00:00, 41.03it/s, loss=0.215, batch#=750]
Testing task 0: 188it [00:02, 63.50it/s, batch#=188]                         
Training task 0, epoch 5: 100%|██████████| 750/750 [00:18<00:00, 41.01it/s, loss=0.208, batch#=750]
Testing task 0: 188it [00:02, 63.93it/s, batch#=188]                         
Training task 0, epoch 6: 100%|██████████| 750/750 [00:18<00:00, 41.01it/s, loss=0.201, batch#=750]
Testing ta

Training task 2, epoch 6: 100%|██████████| 750/750 [00:18<00:00, 41.44it/s, loss=0.234, batch#=750]
Testing task 2: 188it [00:02, 64.02it/s, batch#=188]                         
Testing task 0: 188it [00:02, 63.21it/s, batch#=188]                         
Testing task 1: 188it [00:02, 63.27it/s, batch#=188]                         
Training task 2, epoch 7: 100%|██████████| 750/750 [00:18<00:00, 41.05it/s, loss=0.229, batch#=750]
Testing task 2: 188it [00:02, 64.14it/s, batch#=188]                         
Testing task 0: 188it [00:02, 64.35it/s, batch#=188]                         
Testing task 1: 188it [00:02, 64.51it/s, batch#=188]                         
Training task 2, epoch 8: 100%|██████████| 750/750 [00:18<00:00, 41.13it/s, loss=0.224, batch#=750]
Testing task 2: 188it [00:02, 64.61it/s, batch#=188]                         
Testing task 0: 188it [00:03, 63.30it/s, batch#=188]                         
Testing task 1: 188it [00:03, 62.34it/s, batch#=188]                        

Testing task 1: 188it [00:02, 64.93it/s, batch#=188]                         
Testing task 2: 188it [00:02, 65.16it/s, batch#=188]                         
Training task 3, epoch 13: 100%|██████████| 750/750 [00:18<00:00, 40.99it/s, loss=0.213, batch#=750]
Testing task 3: 188it [00:02, 64.94it/s, batch#=188]                         
Testing task 0: 188it [00:03, 62.10it/s, batch#=188]                         
Testing task 1: 188it [00:02, 63.65it/s, batch#=188]                         
Testing task 2: 188it [00:02, 63.17it/s, batch#=188]                         
Training task 3, epoch 14: 100%|██████████| 750/750 [00:18<00:00, 41.16it/s, loss=0.211, batch#=750]
Testing task 3: 188it [00:02, 64.86it/s, batch#=188]                         
Testing task 0: 188it [00:02, 64.61it/s, batch#=188]                         
Testing task 1: 188it [00:02, 65.66it/s, batch#=188]                         
Testing task 2: 188it [00:02, 65.29it/s, batch#=188]                         
Training task 3, e

Testing task 3: 188it [00:02, 71.48it/s, batch#=188]                         
Training task 4, epoch 15: 100%|██████████| 750/750 [00:15<00:00, 46.91it/s, loss=0.179, batch#=750]
Testing task 4: 188it [00:02, 75.31it/s, batch#=188]                         
Testing task 0: 188it [00:02, 75.08it/s, batch#=188]                         
Testing task 1: 188it [00:02, 74.48it/s, batch#=188]                         
Testing task 2: 188it [00:02, 75.55it/s, batch#=188]                         
Testing task 3: 188it [00:02, 74.47it/s, batch#=188]                         
Training task 5, epoch 1: 100%|██████████| 750/750 [00:15<00:00, 47.28it/s, loss=0.283, batch#=750]
Testing task 5: 188it [00:02, 73.27it/s, batch#=188]                         
Testing task 0: 188it [00:02, 72.55it/s, batch#=188]                         
Testing task 1: 188it [00:02, 72.84it/s, batch#=188]                         
Testing task 2: 188it [00:02, 72.25it/s, batch#=188]                         
Testing task 3: 188

Testing task 0: 188it [00:02, 74.76it/s, batch#=188]                         
Testing task 1: 188it [00:02, 73.82it/s, batch#=188]                         
Testing task 2: 188it [00:02, 74.84it/s, batch#=188]                         
Testing task 3: 188it [00:02, 73.60it/s, batch#=188]                         
Testing task 4: 188it [00:02, 74.48it/s, batch#=188]                         
Training task 5, epoch 15: 100%|██████████| 750/750 [00:15<00:00, 46.69it/s, loss=0.176, batch#=750]
Testing task 5: 188it [00:02, 74.30it/s, batch#=188]                         
Testing task 0: 188it [00:02, 72.57it/s, batch#=188]                         
Testing task 1: 188it [00:02, 72.11it/s, batch#=188]                         
Testing task 2: 188it [00:02, 72.83it/s, batch#=188]                         
Testing task 3: 188it [00:02, 72.45it/s, batch#=188]                         
Testing task 4: 188it [00:02, 71.43it/s, batch#=188]                         
Training task 6, epoch 1: 100%|██████████

Testing task 6: 188it [00:03, 60.19it/s, batch#=188]                         
Testing task 0: 188it [00:03, 59.02it/s, batch#=188]                         
Testing task 1: 188it [00:03, 58.84it/s, batch#=188]                         
Testing task 2: 188it [00:03, 58.93it/s, batch#=188]                         
Testing task 3: 188it [00:03, 58.93it/s, batch#=188]                         
Testing task 4: 188it [00:03, 58.54it/s, batch#=188]                         
Testing task 5: 188it [00:02, 65.16it/s, batch#=188]                         
Training task 6, epoch 13: 100%|██████████| 750/750 [00:15<00:00, 46.69it/s, loss=0.124, batch#=750]
Testing task 6: 188it [00:02, 74.07it/s, batch#=188]                         
Testing task 0: 188it [00:02, 75.26it/s, batch#=188]                         
Testing task 1: 188it [00:02, 75.37it/s, batch#=188]                         
Testing task 2: 188it [00:02, 74.87it/s, batch#=188]                         
Testing task 3: 188it [00:02, 73.42it/s, 

Testing task 5: 188it [00:02, 72.34it/s, batch#=188]                         
Testing task 6: 188it [00:02, 72.04it/s, batch#=188]                         
Training task 7, epoch 9: 100%|██████████| 750/750 [00:16<00:00, 44.84it/s, loss=0.16, batch#=750] 
Testing task 7: 188it [00:02, 62.97it/s, batch#=188]                         
Testing task 0: 188it [00:03, 59.26it/s, batch#=188]                         
Testing task 1: 188it [00:03, 57.25it/s, batch#=188]                         
Testing task 2: 188it [00:03, 53.05it/s, batch#=188]                         
Testing task 3: 188it [00:03, 55.56it/s, batch#=188]                         
Testing task 4: 188it [00:03, 54.67it/s, batch#=188]                         
Testing task 5: 188it [00:03, 54.99it/s, batch#=188]                         
Testing task 6: 188it [00:03, 57.04it/s, batch#=188]                         
Training task 7, epoch 10: 100%|██████████| 750/750 [00:19<00:00, 38.00it/s, loss=0.156, batch#=750]
Testing task 7: 188

Testing task 4: 188it [00:02, 68.10it/s, batch#=188]                         
Testing task 5: 188it [00:02, 69.75it/s, batch#=188]                         
Testing task 6: 188it [00:02, 69.27it/s, batch#=188]                         
Testing task 7: 188it [00:02, 66.63it/s, batch#=188]                         
Training task 8, epoch 5: 100%|██████████| 750/750 [00:16<00:00, 44.80it/s, loss=0.148, batch#=750]
Testing task 8: 188it [00:02, 71.26it/s, batch#=188]                         
Testing task 0: 188it [00:02, 71.53it/s, batch#=188]                         
Testing task 1: 188it [00:02, 71.61it/s, batch#=188]                         
Testing task 2: 188it [00:02, 69.26it/s, batch#=188]                         
Testing task 3: 188it [00:02, 68.72it/s, batch#=188]                         
Testing task 4: 188it [00:02, 69.18it/s, batch#=188]                         
Testing task 5: 188it [00:02, 68.96it/s, batch#=188]                         
Testing task 6: 188it [00:02, 68.59it/s, b

Testing task 6: 188it [00:02, 69.13it/s, batch#=188]                         
Testing task 7: 188it [00:02, 68.45it/s, batch#=188]                         
Training task 8, epoch 15: 100%|██████████| 750/750 [00:17<00:00, 45.66it/s, loss=0.107, batch#=750]
Testing task 8: 188it [00:02, 72.29it/s, batch#=188]                         
Testing task 0: 188it [00:02, 70.83it/s, batch#=188]                         
Testing task 1: 188it [00:03, 59.20it/s, batch#=188]                         
Testing task 2: 188it [00:02, 63.28it/s, batch#=188]                         
Testing task 3: 188it [00:02, 66.80it/s, batch#=188]                         
Testing task 4: 188it [00:02, 68.55it/s, batch#=188]                         
Testing task 5: 188it [00:02, 67.91it/s, batch#=188]                         
Testing task 6: 188it [00:02, 68.34it/s, batch#=188]                         
Testing task 7: 188it [00:02, 69.83it/s, batch#=188]                         
Training task 9, epoch 1: 100%|██████████

Testing task 0: 188it [00:03, 57.60it/s, batch#=188]                         
Testing task 1: 188it [00:03, 57.12it/s, batch#=188]                         
Testing task 2: 188it [00:03, 57.50it/s, batch#=188]                         
Testing task 3: 188it [00:03, 57.53it/s, batch#=188]                         
Testing task 4: 188it [00:03, 59.28it/s, batch#=188]                         
Testing task 5: 188it [00:03, 58.63it/s, batch#=188]                         
Testing task 6: 188it [00:03, 58.77it/s, batch#=188]                         
Testing task 7: 188it [00:03, 59.04it/s, batch#=188]                         
Testing task 8: 188it [00:03, 58.28it/s, batch#=188]                         
Training task 9, epoch 10: 100%|██████████| 750/750 [00:18<00:00, 39.65it/s, loss=0.122, batch#=750]
Testing task 9: 188it [00:03, 59.20it/s, batch#=188]                         
Testing task 0: 188it [00:03, 58.89it/s, batch#=188]                         
Testing task 1: 188it [00:03, 57.40it/s, 

In [47]:
n_task = len(results_no_ewt)
tot_epochs = max([len(i['metrics']) for _, i in results_no_ewt.items()])

fig = plt.figure(figsize=(12, 24))
ax = None

#ax = fig.add_subplot(1, 1, 1)

for i, task in enumerate(results_no_ewt.keys()):
    
    ewt = results_ewt[task]
    no_ewt = results_no_ewt[task]

    x = range(tot_epochs-len(ewt['metrics']), tot_epochs)
    
    m_ewt = defaultdict(list)
    m_no_ewt = defaultdict(list)
    
    for item in ewt['metrics']:
        for k, v in item.items():
            m_ewt[k].append(v)

    for item in no_ewt['metrics']:
        for k, v in item.items():
            m_no_ewt[k].append(v)
            
    ax = fig.add_subplot(n_task, 1, i+1, sharex=ax) 
    
    ax.plot(x, m_ewt['accuracy'], label='ewt')
    ax.plot(x, m_no_ewt['accuracy'], label='no ewt')
    ax.set_xticks(range(tot_epochs),minor=False)
    
    ax.set_title("Task {}".format(task))
    ax.legend(loc="lower left")
    
fig.subplots_adjust(hspace=0.01)

NameError: name 'results_ewt' is not defined

<Figure size 864x1728 with 0 Axes>